<a href="https://colab.research.google.com/github/MiM0ulay/RiskMetrics/blob/main/RiskMetrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install nasdaq-data-link


In [2]:
from datetime import date
import numpy as np
import pandas as pd
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objects as go
import nasdaqdatalink
import yfinance as yf

In [65]:
def calculate_btc_risk_metric():
    # Download historical data from Quandl
  df = nasdaqdatalink.get_table("QDL/BCHAIN", api_key='rRQgKH5b9t6Bmb1z81QE', paginate=True)
  df = df[df["code"] == "MKPRU"].reset_index()
  df['date'] = pd.to_datetime(df['date'])
  df.sort_values(by='date', inplace=True)
  df = df[df['value'] > 0]
  # get data thats not in the quandl database
  new_data = yf.download(tickers='BTC-USD', start='2024-01-01', interval='1d', progress=False)
  new_data.reset_index(inplace=True)
  # restructure yf dataframe to match the quandl one
  new_data.rename(columns={'Date': 'date', 'Open': 'value'}, inplace=True)
  new_data = new_data[['date', 'value']]
  df = pd.concat([df, new_data], ignore_index=True)
  df.drop_duplicates(subset='date', keep='first', inplace=True)
  df.sort_values(by='date', inplace=True)
  btcdata = yf.download(tickers='BTC-USD', period='1d', interval='1m', progress=False)
  btcdata.reset_index(inplace=True)
  df = df.dropna(subset=['value'])
  df = df[['date', 'value']]
  diminishing_factor = 0.395
  moving_average_days = 365
  df['MA'] = df['value'].rolling(moving_average_days, min_periods=1).mean().dropna()
  df['Preavg'] = (np.log(df.value) - np.log(df['MA'])) * df.index**diminishing_factor
  df['avg'] = (df['Preavg'] - df['Preavg'].cummin()) / (df['Preavg'].cummax() - df['Preavg'].cummin())
   # Store results
  results = []
  results.append(['BTC-USD', df['date'].iloc[-1], df['value'].iloc[-1], df['MA'].iloc[-1], df['avg'].iloc[-1]])

  return results


In [41]:
def calculate_risk_metric(ticker, start_date):
  # Download data
  df = yf.download(tickers=ticker, start=start_date, interval='1d', progress=False)

  # Debugging: Check the column names

  # Reset the index and flatten column names (if needed)
  df.reset_index(inplace=True)

  # Handle multi-level columns (flatten them if they exist)
  df.columns = [col[0] if isinstance(col, tuple) else col for col in df.columns]

  # Debugging: Check flattened column names

  # Rename columns for consistency
  if 'Open' in df.columns:
      df.rename(columns={'Date': 'date', 'Open': 'value'}, inplace=True)
  elif 'value' not in df.columns:
      raise KeyError("The required columns ('date', 'value') are not found in the DataFrame.")

  # Keep necessary columns
  df = df[['date', 'value']].copy()

  # Sort by date
  df.sort_values(by='date', inplace=True)

  # Calculate moving average
  moving_average_days = 365
  df['MA'] = df['value'].rolling(moving_average_days, min_periods=1).mean()

  # Ensure no NaN values
  df = df.dropna().reset_index(drop=True)

  # Calculate Preavg
  diminishing_factor = 0.395
  df['Preavg'] = (np.log(df['value']) - np.log(df['MA'])) * (np.arange(len(df)) + 1) ** diminishing_factor
  df['avg'] = (df['Preavg'] - df['Preavg'].cummin()) / (df['Preavg'].cummax() - df['Preavg'].cummin())
  price_per_risk = {
    round(risk, 1):round(np.exp(
        (risk * (df['Preavg'].cummax().iloc[-1] - (cummin := df['Preavg'].cummin().iloc[-1])) + cummin) / df.index[-1]**diminishing_factor + np.log(df['MA'].iloc[-1])
    ))
    for risk in np.arange(0.0, 1.0, 0.1)
  }

   # Store results
  results = []
  results.append([ticker, df['date'].iloc[-1], df['value'].iloc[-1], df['MA'].iloc[-1], df['avg'].iloc[-1]])

  return results



In [67]:
cryptos = { 'ETH-USD': '2016-01-01', 'ETH-EUR': '2016-01-01', 'SOL-USD': '2020-01-01', 'SOL-EUR': '2020-01-01', 'BNB-USD': '2017-01-01', 'SUI20947-USD' : '2023-05-01' }
all_results = []
all_results.extend(calculate_btc_risk_metric())
for ticker, start_date in cryptos.items():
  all_results.extend(calculate_risk_metric(ticker, start_date))

# Create a pandas DataFrame
df_results = pd.DataFrame(all_results, columns=['ticker', 'date', 'value', 'MA', 'avg'])

# Display the DataFrame as a table
styled_df = df_results.style.applymap(lambda x: 'color: green' if x < 0.5 else '', subset=['avg'])
styled_df

<ipython-input-67-4cd64c2e750b>:11: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  styled_df = df_results.style.applymap(lambda x: 'color: green' if x < 0.5 else '', subset=['avg'])


,ticker,date,value,MA,avg
0,BTC-USD,2024-12-30 00:00:00,93536.420000,64715.516000,0.537450
1,ETH-USD,2024-12-30 00:00:00,3349.818848,3041.193808,0.498958
2,ETH-EUR,2024-12-30 00:00:00,3211.725830,2810.966126,0.503908
3,SOL-USD,2024-12-30 00:00:00,189.739212,155.100318,0.589800
4,SOL-EUR,2024-12-30 00:00:00,181.917404,143.358024,0.595512
5,BNB-USD,2024-12-30 00:00:00,693.714844,541.762652,0.359541
6,SUI20947-USD,2024-12-30 00:00:00,4.095997,1.647177,0.855235
